In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 12 11:49:08 2019
@author: wangxinyi
"""
import pymysql
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os
import math

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier 
from collections import defaultdict
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from dateutil import parser
from scipy import stats  
from scipy.stats import entropy
from sklearn.ensemble import GradientBoostingClassifier


conn= pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='root123',charset='utf8',db='car_evaluation')

all_data_df=pd.read_sql_query('''
select id,ty,concat(substr(t,1,9),'0:00') as t, max(x) as x,max(y) as y,ROUND(avg(v),2) as v,ROUND(avg(d),0) as d  , concat(max(x),'-',max(y))  as xy
from tianchi_01_train  group by id,ty,concat(substr(t,1,9),'0:00') order by  id,t;
''',conn)
#where id<100

In [2]:
all_data_df.head(2)

,id,ty,t,x,y,v,d,xy
0,0,拖网,1107 12:00:00,6.118352e+06,5.130672e+06,0.0,0.0,6118351.75770008-5130672.32583525
1,0,拖网,1107 12:10:00,6.118352e+06,5.130672e+06,0.0,0.0,6118351.75770008-5130672.32583525


In [3]:
def t_p(t):
    #return t.size
    return (parser.parse(max(t))-parser.parse(min(t))).total_seconds()/t.size

def entropy_wxy(x):
    index_x={}
    for c in x:
        if c in index_x:
            index_x[c]=index_x[c]+1
        else:
            index_x[c]=1
    vv=np.array(list(index_x.values()))    
    return entropy(vv, base=2)
def sub_num(x):
    return np.max(x)-np.min(x)

def distinct_count(xy):
    return len(set(xy))

def model_num(x):
    return np.mean(x.mode())

def month(x):
    #print(type(x))
    xx=[]
    for c in x:
        xx.append(c[0:2])
    return int(pd.Series(xx).mode()[0])

def day(x):
    #print(type(x))
    xx=[]
    for c in x:
        xx.append(c[5:7])
    return int(pd.Series(xx).mode()[0])

def distance_median(xy):
    test_f=[ [float(xx) for xx in  x.split("-")] for x in xy]
    ss=[]
    for i in range(len(test_f)-1):
        s=np.linalg.norm(np.array(test_f[i])-np.array(test_f[i+1]))  
        ss.append(s)
    return np.percentile(ss,0.5)

def per25(x):
    return np.percentile(x,0.25)

def per75(x):
    return np.percentile(x,0.75)

def kmeans_xy(xy):
    return xy.size



all_df=all_data_df.groupby(['id', 'ty']).agg({'x': [np.mean, np.max,np.min,np.median,model_num,sub_num,per25,per75],
                                              'y': [np.mean, np.max,np.min,np.median,model_num,sub_num,per25,per75],
                                              'v':[np.mean,np.std,np.median,entropy_wxy,model_num,distinct_count,per25,per75] ,
                                              'd':[np.mean,np.std,np.median,entropy_wxy,model_num,distinct_count,per25,per75],
                                              't':[t_p,month,day],
                                              'xy':[distinct_count,distance_median]
                                      })

In [4]:
names_x_x=['c_x','min_x','max_x','median_x','model_x','sub_x','per25_x','per75_x',
           'c_y','min_y','max_y','median_y','model_y','sub_y','per25_y','per75_y',
           'avg_v','stdev_v','median_v','entropy_v','model_v','d_v','per25_v','per75_v',
           'avg_d','stdev_d','median_d','entropy_d','model_d','d_d','per25_d','per75_d',
           't','t_month','t_day',
           'd_xy','dm_xy']
names_y='ty'
all_df.columns=names_x_x

all_df=all_df.reset_index()

In [74]:
all_df.to_csv('dddd.csv')

In [5]:
all_df.head(20)

,id,ty,c_x,min_x,max_x,median_x,model_x,sub_x,per25_x,per75_x,...,entropy_d,model_d,d_d,per25_d,per75_d,t,t_month,t_day,d_xy,dm_xy
0,0,拖网,6.119368e+06,6.152038e+06,6.118352e+06,6.118352e+06,6.118352e+06,33686.667453,6.118352e+06,6.118352e+06,...,0.445872,0.0,15.0,0.0,0.00,2.326629e+05,11,2,23,0.000000
1,1,拖网,6.091460e+06,6.102450e+06,6.049472e+06,6.102450e+06,6.102450e+06,52978.013345,6.049476e+06,6.051141e+06,...,3.377653,0.0,106.0,0.0,0.00,2.459564e+05,11,18,138,0.000000
2,2,拖网,6.183009e+06,6.183191e+06,6.182482e+06,6.183090e+06,6.183090e+06,708.835147,6.182482e+06,6.182482e+06,...,5.586796,0.0,95.0,0.0,0.00,4.171507e+05,11,13,4,0.000000
3,3,拖网,5.239913e+06,5.287805e+06,5.228590e+06,5.229843e+06,5.229843e+06,59214.738740,5.228590e+06,5.228590e+06,...,5.086753,0.0,116.0,0.0,0.00,3.054619e+05,11,20,100,0.000000
4,4,围网,7.062005e+06,7.070797e+06,7.049394e+06,7.061679e+06,7.060750e+06,21402.484584,7.049394e+06,7.049394e+06,...,6.076657,0.0,154.0,0.0,0.00,2.361441e+05,11,0,162,0.000000
5,5,拖网,6.489036e+06,6.503900e+06,6.388833e+06,6.496570e+06,6.496570e+06,115066.764852,6.390302e+06,6.394013e+06,...,6.014084,0.0,147.0,0.0,0.00,1.718592e+05,11,1,92,0.000000
6,6,拖网,6.058548e+06,6.079949e+06,6.032505e+06,6.059514e+06,6.069079e+06,47444.023130,6.032889e+06,6.033464e+06,...,7.357425,14.0,195.0,0.0,3.76,6.157345e+06,11,1,367,287.995949
7,7,围网,6.759621e+06,6.986976e+06,6.587996e+06,6.811245e+06,6.589755e+06,398980.613694,6.588165e+06,6.588190e+06,...,5.718537,0.0,140.0,0.0,0.00,1.813638e+05,11,14,198,0.000000
8,8,拖网,6.201370e+06,6.215849e+06,6.181299e+06,6.203411e+06,6.197745e+06,34549.985805,6.181574e+06,6.182052e+06,...,5.956803,40.0,118.0,0.0,1.00,6.328858e+06,11,5,332,159.671227
9,9,拖网,6.102853e+06,6.102853e+06,6.102853e+06,6.102853e+06,6.102853e+06,0.000000,6.102853e+06,6.102853e+06,...,0.000000,0.0,1.0,0.0,0.00,1.593053e+05,11,0,1,0.000000


In [6]:
d = defaultdict(LabelEncoder)
df = all_df.apply(lambda x:d[x.name].fit_transform(x) if type(x[0]) is str or math.isnan(x[0])  else x)

In [7]:
df#.to_csv('dddd.csv')

,id,ty,c_x,min_x,max_x,median_x,model_x,sub_x,per25_x,per75_x,...,entropy_d,model_d,d_d,per25_d,per75_d,t,t_month,t_day,d_xy,dm_xy
0,0,2,6.119368e+06,6.152038e+06,6.118352e+06,6.118352e+06,6.118352e+06,33686.667453,6.118352e+06,6.118352e+06,...,0.445872,0.0,15.0,0.000,0.0000,2.326629e+05,11,2,23,0.000000
1,1,2,6.091460e+06,6.102450e+06,6.049472e+06,6.102450e+06,6.102450e+06,52978.013345,6.049476e+06,6.051141e+06,...,3.377653,0.0,106.0,0.000,0.0000,2.459564e+05,11,18,138,0.000000
2,2,2,6.183009e+06,6.183191e+06,6.182482e+06,6.183090e+06,6.183090e+06,708.835147,6.182482e+06,6.182482e+06,...,5.586796,0.0,95.0,0.000,0.0000,4.171507e+05,11,13,4,0.000000
3,3,2,5.239913e+06,5.287805e+06,5.228590e+06,5.229843e+06,5.229843e+06,59214.738740,5.228590e+06,5.228590e+06,...,5.086753,0.0,116.0,0.000,0.0000,3.054619e+05,11,20,100,0.000000
4,4,1,7.062005e+06,7.070797e+06,7.049394e+06,7.061679e+06,7.060750e+06,21402.484584,7.049394e+06,7.049394e+06,...,6.076657,0.0,154.0,0.000,0.0000,2.361441e+05,11,0,162,0.000000
5,5,2,6.489036e+06,6.503900e+06,6.388833e+06,6.496570e+06,6.496570e+06,115066.764852,6.390302e+06,6.394013e+06,...,6.014084,0.0,147.0,0.000,0.0000,1.718592e+05,11,1,92,0.000000
6,6,2,6.058548e+06,6.079949e+06,6.032505e+06,6.059514e+06,6.069079e+06,47444.023130,6.032889e+06,6.033464e+06,...,7.357425,14.0,195.0,0.000,3.7600,6.157345e+06,11,1,367,287.995949
7,7,1,6.759621e+06,6.986976e+06,6.587996e+06,6.811245e+06,6.589755e+06,398980.613694,6.588165e+06,6.588190e+06,...,5.718537,0.0,140.0,0.000,0.0000,1.813638e+05,11,14,198,0.000000
8,8,2,6.201370e+06,6.215849e+06,6.181299e+06,6.203411e+06,6.197745e+06,34549.985805,6.181574e+06,6.182052e+06,...,5.956803,40.0,118.0,0.000,1.0000,6.328858e+06,11,5,332,159.671227
9,9,2,6.102853e+06,6.102853e+06,6.102853e+06,6.102853e+06,6.102853e+06,0.000000,6.102853e+06,6.102853e+06,...,0.000000,0.0,1.0,0.000,0.0000,1.593053e+05,11,0,1,0.000000


In [8]:
#df=df.where(df.notnull(), 0.0)

names_x=['c_x','min_x','max_x','median_x','model_x','sub_x','per25_x','per75_x',
           'c_y','min_y','max_y','median_y','model_y','sub_y','per25_y','per75_y',
           'avg_v','stdev_v','median_v','entropy_v','model_v','d_v','per25_v','per75_v',
           'avg_d','stdev_d','median_d','entropy_d','model_d','d_d','per25_d','per75_d',
           't','t_month','t_day',
           'd_xy','dm_xy']
x_train = df.loc[:,names_x].values
y_train = df.loc[:,names_y].values

from imblearn.over_sampling import RandomOverSampler
from collections import Counter

ros = RandomOverSampler(random_state=1)
x_train, y_train = ros.fit_sample(x_train, y_train)
sorted(Counter(y_train).items())


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\

[(0, 4361), (1, 4361), (2, 4361)]

In [93]:
x_train

array([[6.11935124e+06, 6.15203843e+06, 6.11835176e+06, ...,
                   nan, 2.30000000e+01, 0.00000000e+00],
       [6.09146039e+06, 6.10244988e+06, 6.04947187e+06, ...,
                   nan, 1.38000000e+02, 0.00000000e+00],
       [6.18301061e+06, 6.18319102e+06, 6.18248219e+06, ...,
                   nan, 4.00000000e+00, 0.00000000e+00],
       ...,
       [5.24022168e+06, 5.28746208e+06, 5.22870046e+06, ...,
                   nan, 9.60000000e+01, 0.00000000e+00],
       [6.37342539e+06, 6.38762489e+06, 6.36501974e+06, ...,
                   nan, 1.71000000e+02, 0.00000000e+00],
       [6.23542746e+06, 6.24031719e+06, 6.20637253e+06, ...,
                   nan, 5.40000000e+01, 0.00000000e+00]])

In [9]:
#rf = RandomForestClassifier(n_estimators=100,min_samples_leaf=2)
#rf.fit(x_train,y_train)
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

from sklearn.model_selection import GridSearchCV

#param_grid ={'learning_rate':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]}#  {'learning_rate': 0.2},  0.9092252958150469)
# param_grid={'n_estimators':range(50,400,10)}  #{'n_estimators': 230}, 0.913342546264734)
#param_grid={'min_samples_leaf':[2,3,4,5,6]}  # {'min_samples_leaf': 3}, 0.9087862987960976)
#param_grid={'max_depth':range(10,100,10)}  #{'max_depth': 30}, 0.9073549975937918)
#param_grid={'max_features':range(4,24)} {'max_features': 20},  0.913316691603557)
param_grid={'min_samples_split':[250],
            'learning_rate':[0.2],
            
           }
cv = ShuffleSplit(n_splits=3, test_size=0.05, random_state=7)
search = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.2,n_estimators=230,min_samples_split=250,min_samples_leaf=3,
                                                             max_depth=30,max_features=20,subsample=0.95,random_state=7), 
                      param_grid=param_grid, cv=cv,scoring='f1_macro')  #'precision_macro', 'recall_macro','f1_macro'  max_features='sqrt'
search.fit( x_train, y_train)

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=7, test_size=0.05, train_size=None),
             error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.2,
                                                  loss='deviance', max_depth=30,
                                                  max_features=20,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=3,
                                                  min_sampl...
                                                  n_estimators=230,
                                                  n_iter_no_change=None,
             

In [10]:
#search.cv_results_
search.best_score_  
#search.grid_scores_,search.best_params_, search.best_score_
search.best_params_, search.best_score_
#({}, 0.9061905170372865)
#({}, 0.9071618506931185)
#({}, 0.9796498389323539)
#({}, 0.9809640657920312)



({'learning_rate': 0.2, 'min_samples_split': 250}, 0.9842102249676704)

In [44]:
best_parameters = search.best_estimator_.get_params()
for param_name in sorted(best_parameters.keys()):
    print('\t%s: %r' % (param_name, best_parameters[param_name]))

	ccp_alpha: 0.0
	criterion: 'friedman_mse'
	init: None
	learning_rate: 0.2
	loss: 'deviance'
	max_depth: 30
	max_features: 20
	max_leaf_nodes: None
	min_impurity_decrease: 0.0
	min_impurity_split: None
	min_samples_leaf: 3
	min_samples_split: 250
	min_weight_fraction_leaf: 0.0
	n_estimators: 230
	n_iter_no_change: None
	presort: 'deprecated'
	random_state: 6
	subsample: 0.95
	tol: 0.0001
	validation_fraction: 0.1
	verbose: 0
	warm_start: False


In [21]:
p_y= search.predict(x_train)
#df['p_ty']=p_y
f1_score(p_y,y_train,average='macro')

1.0

In [40]:
y_train

array([2, 2, 2, ..., 2, 2, 2], dtype=int64)

In [11]:
all_test_df = pd.read_sql_query('''
select id,concat(substr(t,1,9),'0:00') as t, max(x) as x,max(y) as y,ROUND(avg(v),2) as v,ROUND(avg(d),0) as d  , concat(max(x),'-',max(y))  as xy
from tianchi_01_test  group by id,concat(substr(t,1,9),'0:00') order by  id,t''',conn)


test_df=all_test_df.groupby(['id']).agg({'x': [np.mean, np.max,np.min,np.median,model_num,sub_num,per25,per75],
                                              'y': [np.mean, np.max,np.min,np.median,model_num,sub_num,per25,per75],
                                              'v':[np.mean,np.std,np.median,entropy_wxy,model_num,distinct_count,per25,per75] ,
                                              'd':[np.mean,np.std,np.median,entropy_wxy,model_num,distinct_count,per25,per75],
                                              't':[t_p,month,day],
                                              'xy':[distinct_count,distance_median]
                                      })
test_df.columns=names_x_x
test_df=test_df.reset_index()

In [12]:
x_test = test_df.loc[:,names_x].values
test_y= search.predict(x_test)
test_df['p_ty']=test_y
test_df['p_ty_str']=d['ty'].inverse_transform(test_y)
test_y_p= search.predict_proba(x_test)
test_df['proba_0']=test_y_p[:,0]
test_df['proba_1']=test_y_p[:,1]
test_df['proba_2']=test_y_p[:,2]

In [13]:
result_df=test_df[['id','p_ty_str']]
result_df.to_csv("tianchi01_result_0206_GBDT.csv",index=False,header=0)


In [14]:
result_df_p=test_df[['id','p_ty_str','p_ty','proba_0','proba_1','proba_2']]
result_df_p['proba_max']=result_df_p[['proba_0','proba_1','proba_2']].max(axis=1)
result_df_p=result_df_p[['id','p_ty_str','proba_max']]
result_df_p.to_csv("tianchi01_result_p_0206_GBDT.csv",index=False,header=0)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
